In [4]:
import requests as rq 
import pandas as pd
import csv 
from datetime import datetime, timedelta

In [5]:
weather = []
start_date = datetime(2024, 8, 11)
end_date = datetime(2024, 10, 11)
current_date = start_date
while current_date < end_date: 
    # Format the date for the API call
    date_str = current_date.strftime('%Y-%m-%d')
    url =f'http://api.weatherapi.com/v1/current.json?key=f8d6c8b9e73743cd89e24045241209&q=Ho Chi Minh&aqi=no&dt={date_str}'
    rsp = rq.get(url)
    data = rsp.json()
    weather.append(data)
    # day_data = rsp['forecast']['forecastday'][0]['day']
    # weather.append([
    #         date_str,
    #         rsp['location']['name'],
    #         rsp['location']['country'],
    #         rsp['location']['localtime'],
    #         day_data['maxtemp_c'],
    #         day_data['mintemp_c'],
    #         day_data['avgtemp_c'],
    #         day_data['maxwind_mph'],
    #         day_data['totalprecip_mm'],
    #         day_data['uv'],
    #         day_data['condition']['text']
    #     ]) 
    print(weather)
    current_date += timedelta(days=1)

[{'location': {'name': 'Ho Chi Minh City', 'region': '', 'country': 'Vietnam', 'lat': 10.75, 'lon': 106.6667, 'tz_id': 'Asia/Ho_Chi_Minh', 'localtime_epoch': 1728924008, 'localtime': '2024-10-14 23:40'}, 'current': {'last_updated_epoch': 1728923400, 'last_updated': '2024-10-14 23:30', 'temp_c': 27.0, 'temp_f': 80.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 180, 'wind_dir': 'S', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.76, 'precip_in': 0.03, 'humidity': 89, 'cloud': 0, 'feelslike_c': 31.8, 'feelslike_f': 89.2, 'windchill_c': 25.9, 'windchill_f': 78.6, 'heatindex_c': 29.5, 'heatindex_f': 85.0, 'dewpoint_c': 24.3, 'dewpoint_f': 75.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 9.6, 'gust_kph': 15.5}}]
[{'location': {'name': 'Ho Chi Minh City', 'region': '', 'country': 'Vietnam', 'lat': 10.75, 'lon': 106.6667, 'tz_id': 'Asia/Ho_Chi_M

In [25]:
import pandas as pd
import json

# Load JSON data from file
with open('datajsonfile.json', 'r') as f:
    data = json.load(f)

# Convert JSON data to a pandas DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv('data.csv', index=False)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
df = pd.read_csv('weather_data.csv')

# Check for missing data
missing_data = df.isnull().sum()
print("Missing data in each column:")
print(missing_data)

Missing data in each column:
Date                 0
City                 0
Country              0
Local Time           0
Latitude             0
Longitude            0
Max Temp (C)         0
Min Temp (C)         0
Avg Temp (C)         0
Max Wind (mph)       0
Max Wind (kph)       0
Total Precip (mm)    0
Avg Humidity (%)     0
UV Index             0
Condition            0
dtype: int64


In [8]:
# Fill missing data
# Fill missing numerical data with the mean of the column
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    if df[column].isnull().any():
        df[column].fillna(df[column].mean(), inplace=True)

# Fill missing categorical data with the mode of the column
for column in df.select_dtypes(include=['object']).columns:
    if df[column].isnull().any():
        df[column].fillna(df[column].mode()[0], inplace=True)

In [9]:
# Verify that there are no more missing values
missing_data_after = df.isnull().sum()
print("Missing data after filling:")
print(missing_data_after)

Missing data after filling:
Date                 0
City                 0
Country              0
Local Time           0
Latitude             0
Longitude            0
Max Temp (C)         0
Min Temp (C)         0
Avg Temp (C)         0
Max Wind (mph)       0
Max Wind (kph)       0
Total Precip (mm)    0
Avg Humidity (%)     0
UV Index             0
Condition            0
dtype: int64


In [13]:
import requests as rq
import pandas as pd
import csv
from datetime import datetime, timedelta

weather = []
start_date = datetime(2024, 8, 11)
end_date = datetime(2024, 10, 11)
current_date = start_date

while current_date < end_date: 
    # Format the date for the API call
    date_str = current_date.strftime('%Y-%m-%d')
    url = f'http://api.weatherapi.com/v1/current.json?key=f8d6c8b9e73743cd89e24045241209&q=Ho Chi Minh&aqi=no&dt={date_str}'
    rsp = rq.get(url)
    data = rsp.json()
    weather.append(data)
    current_date += timedelta(days=1)

# Load data
df = pd.read_csv('weather_data.csv')

# Step 1: Convert Local Time to datetime format
df['Local Time'] = pd.to_datetime(df['Local Time'])

# Step 2: Drop redundant columns like 'City', 'Country'
df.drop(columns=['City', 'Country'], inplace=True)

# Step 3: Convert Max Wind from mph to kph
df['Max Wind (kph)'] = df['Max Wind (mph)'] * 1.60934

# Step 4: Remove duplicates
df.drop_duplicates(inplace=True)

# Step 5: Create a new 'Status' column based on 'Condition'
rain_conditions = ['Light rain shower', 'Patchy rain possible', 'Patchy light drizzle', 'Patchy light rain with thunder', 'Light drizzle']
df['Status'] = df['Condition'].apply(lambda x: 1 if x in rain_conditions else 0)

# Step 6: Add Avg Temp (F) column converted from Celsius
df['Avg Temp (F)'] = (df['Avg Temp (C)'] * 9/5) + 32  # Celsius to Fahrenheit conversion

# Reorder columns to place 'Status' before 'Condition' and Avg Temp columns together
cols = df.columns.tolist()
status_index = cols.index('Status')
condition_index = cols.index('Condition')
avg_temp_c_index = cols.index('Avg Temp (C)')
avg_temp_f_index = cols.index('Avg Temp (F)')

# Move Status before Condition
cols.insert(condition_index, cols.pop(status_index))

# Move Avg Temp (F) next to Avg Temp (C)
if avg_temp_f_index > avg_temp_c_index:
    cols.insert(avg_temp_c_index + 1, cols.pop(avg_temp_f_index))

df = df[cols]

# Map the 'Status' column for display purposes
df_display = df.copy()
df_display['Status'] = df_display['Status'].map({1: 'Rain', 0: 'No Rain'})

# Identify numerical and categorical columns
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Display results
print("Numerical Columns:", numerical_columns)
print("Categorical Columns:", categorical_columns)

# Display cleaned data with mapped 'Status'
df_display.head()


Numerical Columns: ['Latitude', 'Longitude', 'Max Temp (C)', 'Min Temp (C)', 'Avg Temp (C)', 'Avg Temp (F)', 'Max Wind (mph)', 'Max Wind (kph)', 'Total Precip (mm)', 'Avg Humidity (%)', 'UV Index', 'Status']
Categorical Columns: ['Date', 'Condition']


,Date,Local Time,Latitude,Longitude,Max Temp (C),Min Temp (C),Avg Temp (C),Avg Temp (F),Max Wind (mph),Max Wind (kph),Total Precip (mm),Avg Humidity (%),UV Index,Status,Condition
0,2023-10-16,2024-10-16 11:08:00,10.75,106.6667,26.1,24.2,25.0,77.00,4.3,6.920162,16.68,93,6.0,Rain,Light rain shower
1,2023-10-17,2024-10-16 11:08:00,10.75,106.6667,30.0,24.0,26.3,79.34,6.9,11.104446,11.10,85,6.0,Rain,Light rain shower
2,2023-10-18,2024-10-16 11:08:00,10.75,106.6667,30.0,24.2,26.4,79.52,5.8,9.334172,15.17,87,7.0,Rain,Light rain shower
3,2023-10-19,2024-10-16 11:08:00,10.75,106.6667,31.8,24.4,27.6,81.68,4.7,7.563898,9.66,82,7.0,No Rain,Patchy light rain
4,2023-10-20,2024-10-16 11:08:00,10.75,106.6667,31.2,24.7,27.0,80.60,5.8,9.334172,34.29,86,7.0,Rain,Patchy light rain with thunder
